# Lecture 17 – Collision theory

In [ ]:
%pip install -q ampform plotly sympy

In [ ]:
from __future__ import annotations

import warnings
from typing import Callable

import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import UnevaluatedExpression, implement_doit_method
from ampform.sympy.math import create_expression
from IPython.display import Math
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

_This notebook is an attempt to recreate the Mathematica notebook [provided by Miguel Albaladejo](https://indico.ific.uv.es/event/6803/contributions/21224). Another nice tutorial about the complex plane is [this Julia notebook](https://mmikhasenko.github.io/ReactionAmplitudeExamples.jl/N-029-introduction2complexplane.html) by Mikhail Mikhasenko._

## Riemann sheets

### Square root example

There are multiple solutions for $x$ to the equation $y^2 = x$ – the fact that we usually take $y = \sqrt{x}$ to be the solution to this equation is just a matter of convention. It would be more complete to represent the solution as a set of points in the complex plane. In this case, we have the set $S = \left\{\left(z, w\right)\in\mathbb{C}^2 | w^2=z\right\}$. This is set forms a **Riemann surface** in $\mathbb{C}^2$ space.

In [ ]:
def plot_riemann_surfaces(
    funcs: list[Callable],
    func_unicode: str,
    boundaries: tuple[complex, float] | tuple[complex, complex] = (0, 1),
    T_cutoff: float | None = None,
    resolution: int | tuple[int, int] = 50,
    colorize: bool = True,
    mask: Callable[[np.ndarray, np.ndarray], bool] | None = None,
) -> None:
    X, Y = create_meshgrid(boundaries, resolution)
    Z = X + Y * 1j
    T = [f(Z) for f in funcs]
    if T_cutoff is not None:
        for t in T:
            t[np.abs(t) > T_cutoff] = np.nan
    if mask is not None:
        the_mask = mask(X, Y)
        X[the_mask] = np.nan
        Y[the_mask] = np.nan

    vmax = max(max(t.imag.max(), t.real.max()) for t in T)
    colorscale = [[i, c] for i, c in enumerate(DEFAULT_PLOTLY_COLORS)]
    style = lambda i, t: dict(
        cmin=-vmax,
        cmax=+vmax,
        showscale=colorize,
        colorscale="RdBu_r"
        if colorize
        else [[0, "rgb(0, 0, 0)"], [1, DEFAULT_PLOTLY_COLORS[i - 1]]],
        surfacecolor=t.real if colorize else np.ones(shape=t.shape),
    )
    S_im = [
        go.Surface(x=X, y=Y, z=t.imag, **style(i, t), name=f"Sheet {i}")
        for i, t in enumerate(T, 1)
    ]
    S_re = [
        go.Surface(x=X, y=Y, z=t.real, **style(i, t), name=f"Sheet {i}")
        for i, t in enumerate(T, 1)
    ]
    fig = make_subplots(
        cols=2,
        specs=[[{"type": "surface"}, {"type": "surface"}]],
        subplot_titles=(f"Im {func_unicode}", f"Re {func_unicode}"),
    )
    for i in range(len(funcs)):
        fig.add_trace(S_im[i], col=1, row=1)
        fig.add_trace(S_re[i], col=2, row=1)
    fig.update_layout(height=550, width=1_000)
    fig.update_traces(colorbar=dict(title="Re/Im"))
    fig.show()


def create_meshgrid(
    boundaries: tuple[complex, float] | tuple[complex, complex] = (0, 1),
    resolution: int | tuple[int, int] = 50,
) -> tuple[np.ndarray, np.ndarray]:
    if isinstance(resolution, tuple):
        x_res, y_res = resolution
    else:
        x_res, y_res = resolution, resolution
    box_min, box_max = boundaries
    if isinstance(box_max, (float, int)):
        pos, r_max = box_min, box_max
        R, Θ = np.meshgrid(
            np.linspace(0, r_max, num=x_res),
            np.linspace(-np.pi, +np.pi, num=y_res),
        )
        X = R * np.cos(Θ) + pos
        Y = R * np.sin(Θ) + pos
        return X, Y
    x1 = complex(box_min).real
    x2 = complex(box_max).real
    y1 = complex(box_min).imag
    y2 = complex(box_max).imag
    return np.meshgrid(
        np.linspace(x1, x2, num=x_res),
        np.linspace(y1, y2, num=y_res),
    )

In [ ]:
plot_riemann_surfaces(
    funcs=[lambda z: -np.sqrt(z), lambda z: +np.sqrt(z)],
    func_unicode="±√z",
)

In [ ]:
plot_riemann_surfaces(
    funcs=[
        lambda z: -1 / np.sqrt(z),
        lambda z: +1 / np.sqrt(z),
    ],
    func_unicode="1/±√z",
    T_cutoff=9,
)

Note also that since $y = e^{x + 2n \pi i}$ for $\forall n \in \mathbb{Z}$, we have that $x = \log(y) + 2n\pi i$:

In [ ]:
plot_riemann_surfaces(
    funcs=[
        lambda z: np.log(z) - 2j * np.pi,
        lambda z: np.log(z) + 2j * np.pi,
        np.log,
    ],
    func_unicode="log z",
    boundaries=(0, np.e**2),
    T_cutoff=10,
)

<details>
<summary>Video explainers</summary>

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/R9MX8QgKwtg?si=kK-_1Po4XHzRpzR9" title="What are... Riemann surfaces? [VisualMath]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

<iframe width="560" height="315" src="https://www.youtube.com/embed/sD0NjbwqlYw?si=XQMBxuwolPGlVbwq" title="Visualizing the Riemann zeta function and analytic continuation [3Blue1Brown]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
  
</details>

## Definition of the G(s) functions

In [ ]:
@implement_doit_method
class SignedSqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, z, sign, **hints) -> SignedSqrt:
        return create_expression(cls, z, sign, **hints)

    def evaluate(self) -> sp.Expr:
        z, sign = self.args
        return sp.sqrt(abs(z)) * sp.exp(sign * sp.I * sp.arg(z) / 2)

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        sign = _render_sign(printer, self.args[1])
        return Rf"\sqrt[{sign}]{{{z}}}"


def _render_sign(printer, sign) -> str:
    if sign == +1:
        return "+"
    if sign == -1:
        return "-"
    return printer._print(sign)


z = sp.Symbol("z", complex=True)
neg_sqrt = SignedSqrt(z, sign=-1)
pos_sqrt = SignedSqrt(z, sign=+1)
Math(aslatex({e: e.doit() for e in [neg_sqrt, pos_sqrt]}))

In [ ]:
plot_riemann_surfaces(
    funcs=[sp.lambdify(z, pos_sqrt.doit())],
    func_unicode="⁺√z",
)

In [ ]:
@implement_doit_method
class G(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m, g0, sign=+1, **hints) -> Sigma:
        return create_expression(cls, s, m, g0, sign, **hints)

    def evaluate(self) -> sp.Expr:
        s, m, g0, sign = self.args
        sigma = Sigma(s, m)
        g = (g0 - sigma * sp.log((sigma - 1) / (sigma + 1))) / (16 * sp.pi**2)
        return sp.Piecewise(
            (g, sign < 0),
            (G(s, m, g0, sign=-1) + 2 * sp.I * sigma / (16 * sp.pi), True),
        )

    def _latex(self, printer, *args) -> str:
        s = printer._print(self.args[0])
        sign = self.args[-1]
        number = "I" if sign < 0 else "II"
        return f"G_{{{number}}}({s})"


@implement_doit_method
class Sigma(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m, **hints) -> Sigma:
        return create_expression(cls, s, m, **hints)

    def evaluate(self) -> sp.Expr:
        s, m = self.args
        return sp.sqrt(1 - 4 * m**2 / s)

    def _latex(self, printer, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\sigma\left({s}\right)"


s, g0 = sp.symbols("s g0", complex=True)
m = sp.Symbol("m", real=True, nonnegative=True)
sigma = Sigma(s, m)
G1 = G(s, m, g0, sign=-1)
G2 = G(s, m, g0, sign=+1)
definitions = {e: e.doit(deep=False) for e in [G1, G2, sigma]}
Math(aslatex(definitions))

In [ ]:
substitutions = {
    m: 139,
    g0: 3.0,
}
Math(aslatex(substitutions))

In [ ]:
G1_expr = G1.doit().xreplace(substitutions).xreplace(substitutions)
G2_expr = G2.doit().xreplace(substitutions).xreplace(substitutions)
assert G1_expr.free_symbols == {s}
assert G2_expr.free_symbols == {s}
G1_func = sp.lambdify(s, G1_expr)
G2_func = sp.lambdify(s, G2_expr)

In [ ]:
plot_riemann_surfaces(
    funcs=[
        lambda z: G1_func(z**2),
        lambda z: G2_func(z**2),
    ],
    func_unicode="G(s)",
    boundaries=(240 - 40j, 320 + 40j),
    colorize=False,
    resolution=(20, 301),
    mask=lambda x, y: np.abs(y) < 1e-6,
)